<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Удаление-лишнего-столбца" data-toc-modified-id="Удаление-лишнего-столбца-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Удаление лишнего столбца</a></span></li><li><span><a href="#Выявление-дубликатов" data-toc-modified-id="Выявление-дубликатов-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Выявление дубликатов</a></span></li><li><span><a href="#Ограничение-размера-датасета" data-toc-modified-id="Ограничение-размера-датасета-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Ограничение размера датасета</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Предобработка данных</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Определение токсичности комментариев

Заказчик: интернет-магазин «Викишоп».

Требуется создание модели для выявления токсичных комментариев для последующей отправки их на модерацию.
Требование заказчика: качество модели необходимо оценивать по метрике F1 значение которой должно быть не меньше 0,75.

**Описание данных**

Файл: `toxic_comments.csv`.
 - *text*  - содержит текст комментария.
 - *toxic* - целевой признак (0 - нормальный комментарий, 1 - токсичный).

## План работы

- Загрузка необходимых библиотек
- Загрузка данных и их анализ
    - Проверка наличия пропусков
    - Проверка наличия дубликатов
- Предобработка данных
    - Лемматизация и очистка
    - Рассчет TF-IDF
- Обучение нескольких моделей и выбор лучшей


## Подготовка

### Загрузка данных

In [1]:
#установка недостающих библиотек  и недостающих обновлений в окружение
!pip install -U scikit-learn -q
!pip install -U spacy==3.2.0 -q

In [2]:
#импорт библиотек и определение констант
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords as nltk_stopwords
import spacy
import re 
from lightgbm import LGBMClassifier

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import (
    train_test_split, 
    RandomizedSearchCV
)

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score 

In [3]:
RANDOM_STATE = 42

In [4]:
#загрузка файла toxic_comments.csv
data = pd.read_csv('/datasets/toxic_comments.csv')
display(data.head(10))
data.info()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


**Вывод**

Файл toxic_comments.csv
 - число строк: 159 292
 - число столбцов: 3
 - пустые значения: нет 
 - типы данных: соответствуют данным в файле
 - переименование столбцов: не требуется
    
Один из столбцов требует удаления (не указанн в аннотации к файлу, содержит копию индексов)

### Удаление лишнего столбца

In [5]:
# удаление "лишнего" столбца
data = data.drop('Unnamed: 0', axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


**Вывод**

Удаление проведено успешно.

### Выявление дубликатов

In [6]:
# выявление явных дубликатов
data.duplicated().sum()

0

In [7]:
# выявление неявных дубликатов
data['text'].duplicated().sum()

0

**Вывод**

Дубликаты не выявлены

### Ограничение размера датасета

In [8]:
# подсчет доли токсичных записей в датасете
data['toxic'].sum()/data.shape[0]

0.10161213369158527

В связи с ограничениями вычислительных мощностей, необходимо ограничить объем данных с сохранением дисбаланса классов для дальнейшей работы.

In [9]:
# создание тестовой выборки с сохранением дисбаланса классов
data_short = data.groupby('toxic').apply(lambda x: x.sample(frac=0.5, random_state=RANDOM_STATE)).reset_index(drop=True)
data_short.head()

,text,toxic
0,"Press \n June 2011: Today Show, NBC: Today's M...",0
1,Contested deletion \n\nThis article should not...,0
2,"""\n Your latest edits have goen even further ...",0
3,"""\nIt should not be deleted, but fixed. North ...",0
4,I'm back... \n\n...I haven't found the rusty k...,0


In [10]:
data_short.toxic.value_counts()

0    71553
1     8093
Name: toxic, dtype: int64

**Вывод** 
В качестве датасета для текущей задачи был взять фрагмент исходного датасета размером 4000 записей с сохранением исходного дисбаланса классов.

### Предобработка данных

In [11]:
# создание модели для лемматизации
load_model = spacy.load('en_core_web_sm', disable = ['parser','ner'])

In [12]:
def lemmat(text, model=load_model):
    """
    функция для лемматизации английского текста. 
    На вход получает список текстов и задается по умолчанию модель для лемматизации.
    на выходе функция возвращает список лемматизированного почищенного текста (оставляет символы только английского алфавита)
    """
    lemm = model(text)
    lemm = " ".join(re.sub(r'[^a-zA-Z ]', " ", str(lemm)).split()) #избавление от лишних символов и объединенеие всех слов
    
    return lemm

In [13]:
# лемматизация  датасета
data_short['text'] = data_short['text'].apply(lemmat)

In [14]:
# создание переменной со стоп-словами
nltk.download('stopwords')
stopwords = list(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
# деление на тренировочные и тестовые выборки
#разбиение датасета на тренировочне и тестовые выборки
X = data_short.drop('toxic', axis=1)
y = data_short['toxic']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE, stratify=y)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(59734, 1)
(19912, 1)
(59734,)
(19912,)


In [16]:
# рассчет tf-idf (обучение) на лемматизированных тренировочных данных
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 

X_train = count_tf_idf.fit_transform(X_train['text'])

print(X_train.shape)

(59734, 95531)


In [17]:
# рассчет tf-idf на лемматизированных тестовых данных
X_test = count_tf_idf.transform(X_test['text'])

print(X_test.shape)

(19912, 95531)


**Вывод**

Для дальнейшего обучения и тестирования модели машинного обучения была проведена лемматизация и произведен рассчет TF-IDF.

## Обучение

In [18]:
# итоговый пайплайн: подготовка данных и модель
pipe_final = Pipeline(
    [
        ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))
    ]
) 

In [19]:
#параметры для кросс-валидации

param_distributions = [
     словарь для модели DecisionTreeRegressor()
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        'models__max_depth': range(24, 25),
        'models__min_samples_split': range(8, 9),
        'models__min_samples_leaf': range(8, 9)
    },
     словарь для модели LogisticRegression()
    {
        'models': [LogisticRegression()],
        'models__penalty': [None],
        'models__C': [7]
    },
     словарь для модели LGBMClassifier()
    {
        'models': [LGBMClassifier(random_state=RANDOM_STATE)],
        'models__boosting_type': ['gbdt'],
        'models__max_depth': range(24, 25),
        'models__n_estimators' : [200]
    },
     словарь для модели KNeighborsClassifier()
    {
        'models': [KNeighborsClassifier(n_neighbors=50)]
    }
]

In [20]:
#кросс-валидация
search_cv = RandomizedSearchCV(
    pipe_final, 
    param_distributions, 
    cv=5, 
    scoring='f1', 
    n_jobs=-1,
    error_score='raise',
    random_state=RANDOM_STATE
)

In [21]:
#обучение модели кросс-валидации
search_cv.fit(X_train, y_train);

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_search.py:318: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1186: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1186: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1186: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1186: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1186: 

In [22]:
#вывод метрики для лучшей модели на теринировочной выборке
print ('Метрика лучшей модели на тренировочной выборке:', search_cv.best_score_)

# предсказание на тестовых данных и вывод метрики для предсказания
y_test_pred = search_cv.predict(X_test)
print(f'Метрика f1 на тестовой выборке: {f1_score(y_test, y_test_pred)}')

Метрика лучшей модели на тренировочной выборке: 0.7532680275431878
Метрика f1 на тестовой выборке: 0.7612529613056067


In [23]:
#вывод лучшей модели
print('Лучшая модель и её параметры:\n\n', search_cv.best_estimator_)

Лучшая модель и её параметры:

 Pipeline(steps=[('models', LogisticRegression(C=7, penalty=None))])


**Вывод**

Был собран пайплайн, проведена кросс-валидация и оценено предсказание на обученной модели.

В результате лучшей моделью стала: LogisticRegression() со следующими параметрами: C=7, penalty=None

На модели были получены следующие метрики:
- Метрика f1 лучшей модели на тренировочной выборке: 0.753
- Метрика f1 на тестовой выборке: 0.761

## Выводы

От заказчика получен датасет toxic_comments.csv, который содержит 159 292 записей. В связи с невозможностью обрабатывать большие объемы текстов, пришлось сократить количество записей в датасете для обучения и тестирования до 4000. 

Была проведена лемматизация и очистка данных с последующим удалением образовавшихся дубликатов

Был собран пайплайн, проведена кросс-валидация и оценено предсказание на обученной модели.

В результате лучшей моделью была выбрана: LogisticRegression() со следующими параметрами: C=7, penalty=None

На модели были получены следующие метрики:
- Метрика f1 лучшей модели на тренировочной выборке: 0.753
- Метрика f1 на тестовой выборке: 0.761